In [31]:
# https://www.time.ir/
# تکرار جست و جو تا زمانی که یک سایت کارت تکراری پیدا کند 

In [7]:
import subprocess

subprocess.Popen("C:\Program Files\Cloudflare\Cloudflare WARP\Cloudflare WARP.exe")

<Popen: returncode: None args: 'C:\\Program Files\\Cloudflare\\Cloudflare WA...>

In [45]:
import requests
from bs4 import BeautifulSoup
import cv2
import pytesseract
from matplotlib import pyplot as plt
from selenium.webdriver.common import keys
from selenium import webdriver
import sqlite3
import time
import datetime
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from jalali import *
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select


# security_captcha to text
def imagetostr(imagepath):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
    img = cv2.imread(imagepath)
    ret, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
    text = pytesseract.image_to_string(img).replace("\n", "").replace(' ', '''!()-[]{};:'"\,<>./?@#$%^&*_~''')
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


# find img
def find_ca_im(driver):
    # get captcha image
    security_captcha_imgs = driver.find_elements_by_tag_name('img')
    for item in security_captcha_imgs:
        htmlout = item.get_attribute("outerHTML")
        captchalist = ["capcha", "captcha", "security", "security_captcha", "securitycaptcha"]
        if len([ii for ii in captchalist if ii in htmlout]) >= 1:
            security_captcha_img = item
            actions = ActionChains(driver)
            actions.move_to_element(security_captcha_img)
            actions.perform()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    with open('security_captcha.png', 'wb') as file:
        file.write(security_captcha_img.screenshot_as_png)
    ckeys = imagetostr('security_captcha.png')
    return ckeys


# popup - splash-view
def rm_popup(driver):
    try:
        splash_container = driver.find_element_by_class_name('splash-close-button')
        splash_container.click()
    except:
        pass
    try:
        buttons = driver.find_elements_by_tag_name("button")
        for i in buttons:
            htmlout = i.get_attribute("outerHTML")
            if "ادامه" in htmlout:
                i.click()
    except:
        pass

    try:
        buttons = driver.find_elements_by_tag_name("button")
        for i in buttons:
            htmlout = i.get_attribute("outerHTML")
            if "بستن" in htmlout:
                i.click()
    except:
        pass


# singup
def singupform(driver,mobile_num,email,username,password):
    try:
        Mobile_num_label = driver.find_element_by_xpath("//*[text()='شماره تلفن همراه']/..")
        Mobile_num = Mobile_num_label.find_element_by_tag_name("input")
        data_format = Mobile_num.get_attribute('data-format')
        if mobile_num[0] == '0':
            mobile_num = mobile_num.replace("0", "", 1)
        if data_format == '++(+++)+++++++':
            mobile_num = '98' + mobile_num
        Mobile_num.clear()
        Mobile_num.click()
        Mobile_num.send_keys(mobile_num)
    except:
        pass

    #     try:
    #         Terms = driver.find_element_by_id('terms')
    #         Terms.click()
    #     except:
    #         pass

    try:
        Email_label = driver.find_element_by_xpath("//*[text()='ایمیل']/..")
        Email = Email_label.find_element_by_tag_name("input")
        Email.clear()
        Email.send_keys(email)
    except:
        pass

    try:
        Usename_label = driver.find_element_by_xpath("//*[text()='نام کاربری']/..")
        Usename = Usename_label.find_element_by_tag_name("input")
        Usename.clear()
        Usename.send_keys(username)
    except:
        pass

    try:
        Fname_label = driver.find_element_by_xpath("//*[text()='نام و نام خانوادگی']/..")
        Fname = Fname_label.find_element_by_tag_name("input")
        Fname.clear()
        Fname.send_keys(username)
    except:
        pass

    try:
        Password_label = driver.find_element_by_xpath("//*[text()='رمز عبور']/..")
        Password = Password_label.find_element_by_tag_name("input")
        Password.clear()
        Password.send_keys(password)
    except:
        pass

    try:
        Confirm_password_label = driver.find_element_by_xpath("//*[text()='تکرار رمز عبور']/..")
        Confirm_password = Confirm_password_label.find_element_by_tag_name("input")
        Confirm_password.clear()
        Confirm_password.send_keys(password)
    except:
        pass

    try:
        Captcha_label = driver.find_element_by_xpath("//*[text()='کد امنیتی']/..")
        Captcha = Captcha_label.find_element_by_tag_name("input")
        cod = find_ca_im(driver)
        Captcha.clear()
        Captcha.send_keys(cod)
    #         Captcha.send_keys(keys.Keys.ENTER)
    except:
        pass


def singup(driver,mobile_num,email,username,password):
    rm_popup(driver)
    singupurl = driver.find_element_by_xpath("//*[text()='ثبت نام']").get_attribute("href")
    driver.get(singupurl)
    c_url = driver.current_url
    issing = 1
    while (c_url == driver.current_url) and issing :
        rm_popup(driver)
        singupform(driver,mobile_num,email,username,password)
#         time.sleep(3)
        buttons = driver.find_elements_by_tag_name("a")
        for button in buttons:
            btntxt = button.get_attribute('outerHTML')
            if btntxt.find("ثبت نام") >= 0 and btntxt.find('form-button') >= 0:
                button.click()
                time.sleep(2)
        singup_stoplist = ['ایمیل شما قبلا', 'نام کاربری (ایمیل) موجود می باشد']
        body = driver.find_element_by_tag_name('body').get_attribute('outerHTML')
        for item in singup_stoplist:
            issing = body.find(item)
            if issing > 0:
                cur.execute("update betscan_website set singin = 1 where address = '{}' ".format(address))
                con.commit()
                issing = 0
                break
            
    rm_popup(driver)
    if issing == 0 :   
        singin(driver,mobile_num,email,username,password)


def singin(driver,mobile_num,email,username,password):
    rm_popup(driver)
    a_urls = driver.find_elements_by_tag_name('a')
    for item in a_urls:
        try:
            item_txt = item.get_attribute('outerHTML')
            if item_txt.find('ورود') >= 0:
                driver.get(item.get_attribute('href'))
        except:
            pass
    c_url = driver.current_url
    while c_url == driver.current_url:
        singupform(driver,mobile_num,email,username,password)
        time.sleep(3)
        driver.find_element_by_tag_name('input').send_keys(keys.Keys.ENTER)

    rm_popup(driver)


In [88]:
# set defult value

website_url = 'https://salammati.online/'  #
group = 'A'

if 'https://' in website_url :
    pass
else : 
    website_url = 'https://' + website_url
    
address = website_url
creat_at = datetime.datetime.now()
username = 'userbet'
fname = 'jamshidibet'
password = 'Pass123456'
mobile_num = '9380084250'
email = "alirezabet72_gmail@gmail.com"

# cart
cart_id = '5022291310514847'
cart_pass2 = '4241930'
cart_cvv2  = '185'
cart_month = '12'
cart_year = '1405'

#
Payment_gateway = ""


In [86]:
# time.ir
# driver = webdriver.Chrome(executable_path="chromedriver.exe")
# driver.get('https://www.time.ir/')
# driver.set_window_position(-7,0)
# driver.set_window_size(697,735)


# s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(executable_path="chromedriver.exe")
# driver = webdriver.Chrome(service=s)
driver.set_window_position(676,0)
driver.set_window_size(697,735)
try:
    driver.get(website_url)
    title = driver.title
except:
    print('get erorr')

C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3602266790.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver.exe")


In [90]:

con = sqlite3.connect('db.sqlite3')
cur = con.cursor()

rm_popup(driver)
# check if exsist in db
res = cur.execute('SELECT * FROM betscan_website where address = "{}"'.format(address))
if len(res.fetchall()) <= 0:
    cur.execute(
        "insert into betscan_website (address, creat_at, password, sgroup, username, singin, title) values ('{}','{}','{}','{}','{}','{}','{}')".format(
            address,creat_at,password,group,username,0,title))
    con.commit()

# check singin
res = cur.execute('SELECT singin FROM betscan_website where address = "{}"'.format(address))

try:
    rm_popup(driver)
    if int(res.fetchone()[0]) == 0:
        rm_popup(driver)
        singup(driver,mobile_num,email,username,password)
        rm_popup(driver)
    else:
        singin(driver,mobile_num,email,username,password)
        rm_popup(driver)
        cur.execute("update betscan_website set singin = 1 where address = '{}' ".format(address))
        con.commit()
except:
    pass

con.close()


try:
    rm_popup(driver)
    links = driver.find_elements_by_tag_name('a')
    for link in links:
        htmlout = link.get_attribute("outerHTML")
        txtlist = ['شارژ', 'افزایش موجودی', 'موجودی','شارژ حساب']
        for item in txtlist:
            if htmlout.find(item) >= 0:
                driver.get(link.get_attribute("href"))
except:
    pass

rm_popup(driver)

try:
    links = driver.find_elements_by_tag_name('a')
    rm_popup(driver)
    for link in links:
        htmlout = link.get_attribute("outerHTML")
        dargatext = ['وداپى',"درگاه مستقیم", "درگاه انلاین", "درگاه آنلاین(واریز از مبلغ 100 هزار تومان)", 'مستقيم', 'بانکی']
        for text in dargatext:
            if htmlout.find(text) >= 0:
                driver.get(link.get_attribute("href"))
except:
    pass

rm_popup(driver)

time.sleep(3)

c_url = driver.current_url

try:
    while c_url == driver.current_url:
        time.sleep(3)
        try:
            Captcha_label = driver.find_element_by_xpath("//*[text()='کد امنیتی']/..")
            Captcha = Captcha_label.find_element_by_tag_name("input")
            cod = find_ca_im(driver)
            Captcha.clear()
            Captcha.send_keys(cod)
        except:
            print('error Captcha')
        Captcha_label = driver.find_element_by_xpath("//*[text()='کد امنیتی']/../..")
        Captcha_label.find_element_by_xpath("..//*[text()='شارژ حساب']").click()
except:
    pass


C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\1644148324.py:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  splash_container = driver.find_element_by_class_name('splash-close-button')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\1644148324.py:60: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\1644148324.py:69: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\1644148324.py:153: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  singupurl = driver.find_element_by_xpath("//*[text()='ثبت نام']").get_attribute("href")
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\5991278

error Captcha


# درگاه

In [91]:
Payment_gateway = driver.title
Payment_gateway

'Payment'

# Payment

In [96]:
try:
    # cart
    Input = driver.find_element_by_xpath('//*[@id="vs-input--card-number"]')
    Input.clear()
    Input.send_keys(cart_id)
except:
    pass

try:
    # phone
    Input = driver.find_element_by_xpath('//*[@id="vs-input--card-phone_number"]')
    Input.clear()
    Input.send_keys('0'+mobile_num)
except:
    pass

try:
    Input = driver.find_element_by_xpath('//*[@id="q-app"]/div/div/main/div/div[2]/div/div/div[1]/div[3]/div/div[3]/button')
    Input.click()
except:
    pass

try:
    smscod = input()
    Input = driver.find_element_by_xpath('//*[@id="vs-input--verification-code"]')
    Input.clear()
    Input.send_keys(smscod)
    driver.find_element_by_xpath('/html/body/div[2]/div/footer/div/div/div[3]/button').click()
except:
    pass


C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3999049923.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="vs-input--card-number"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3999049923.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="vs-input--card-phone_number"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3999049923.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="q-app"]/div/div/main/div/div[2]/div/div/div[1]/div[3]/div/div[3]/button')


53312


C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3999049923.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="vs-input--verification-code"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3999049923.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[2]/div/footer/div/div/div[3]/button').click()


In [97]:
try:
    # cvv
    Input = driver.find_element_by_xpath('//*[@id="vs-input--card-cvv2"]')
    Input.clear()
    Input.send_keys(cart_cvv2)
except:
    pass


try:
    # mah sal
    Input1 = driver.find_element_by_xpath('//*[@id="vs-input--card-month"]')
    Input2 = driver.find_element_by_xpath('//*[@id="vs-input--card-year"]')
    Input1.clear()
    Input1.send_keys(cart_month)
    Input2.clear()
    Input2.send_keys(cart_year[2:])
except:
    pass


try:
    # pass2
    Input = driver.find_element_by_xpath('//*[@id="vs-input--138"]')
    Input.clear()
    Input.send_keys(cart_pass2)
except:
    pass

try:
    Input = driver.find_element_by_xpath('//*[@id="q-app"]/div/div/main/div/div[2]/div/div/div[1]/div[3]/div/div[3]/button')
    Input.click()
except:
    pass
time.sleep(30)
try:
    bodytxt = driver.find_element_by_tag_name('body').get_attribute('outerHTML')
    if bodytxt.find('کافی نیست') >=0 :
        driver.quit()
except:
    pass

C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3826074286.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="vs-input--card-cvv2"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3826074286.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input1 = driver.find_element_by_xpath('//*[@id="vs-input--card-month"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3826074286.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input2 = driver.find_element_by_xpath('//*[@id="vs-input--card-year"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3826074286.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="vs-input--138"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\38

# Parscoinpay 

In [114]:
Payment_gateway = driver.title

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("متوجه شدم") >=0 :
            button.click()
except:
    pass

try:
    # cart
    Input = driver.find_element_by_xpath('/html/body/div/div[2]/div[1]/div[2]/div/div[1]/input')
    Input.clear()
    Input.send_keys(cart_id)
except:
    pass

try:
    # phone
    Input = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[1]/div[3]/div/input')
    Input.clear()
    Input.send_keys(mobile_num)
except:
    pass

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("پرداخت") >=0 :
            button.click()
except:
    pass

C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\1965640876.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")


In [104]:
#cod
try:
    # mli
    Input = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[1]/div/section/div/input')
    Input.clear()
    Input.send_keys('861980')
except:
    pass

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("تایید") >=0 :
            button.click()
except:
    pass

C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\341388497.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[1]/div/section/div/input')
C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\341388497.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")


In [116]:
try:
    # cvv
    Input = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[3]/div[2]/input')
    Input.clear()
    Input.send_keys(cart_cvv2)
except:
    pass


try:
    # mah sal
    Input1 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[4]/div[2]/div[1]/input')
    Input2 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[4]/div[2]/div[2]/input')
    Input1.clear()
    Input1.send_keys(cart_month)
    Input2.clear()
    Input2.send_keys(cart_year[2:])
except:
    pass


try:
    # pass2
    Input = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[5]/div[2]/input')
    Input.clear()
    Input.send_keys(cart_pass2)
except:
    pass

try:
    # pass2
    Input = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[6]/div[1]')
    Input.click()
except:
    pass



C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\2505365071.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[3]/div[2]/input')
C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\2505365071.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input1 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[4]/div[2]/div[1]/input')
C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\2505365071.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Input2 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div/div/div[2]/div[4]/div[2]/div[2]/input')
C:\Users\farhad\AppData\Local\Temp\ipykernel_10280\2505365071.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_elemen

In [67]:
Payment_gateway = driver.title
print(Payment_gateway)
try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("متوجه شدم") >=0 :
            button.click()
except:
    pass

try:
    # cart
    Inputs = driver.find_elements_by_tag_name("input")
    for elemt in Inputs :
        elemt_txt = elemt.get_attribute('outerHTML')
        if elemt_txt.find("v-card-number") >=0 :
            Input = elemt
            Input.clear()
            Input.send_keys(cart_id)
except:
    pass

try:
    # mobile_num
    for elemt in Inputs :
        elemt_txt = elemt.get_attribute('outerHTML')
        if elemt_txt.find("v-mobile-number") >=0 :
            Input = elemt
            Input.clear()
            Input.send_keys('0'+mobile_num)
except:
    pass

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("مرحله بعد") >=0 :
            button.click()
except:
    pass

time.sleep(3)
try:
    Inputs = driver.find_elements_by_tag_name("input")
    for elemt in Inputs :
        elemt_txt = elemt.get_attribute('outerHTML')
        if elemt_txt.find("v-card-pin-2") >=0 :
            Input = elemt
            Input.clear()
            Input.send_keys(cart_pass2)
except:
    pass

try:
    # cart cvv2
    Input = driver.find_element_by_id("v-card-cvv")
    Input.clear()
    Input.send_keys(cart_cvv2)
except:
    pass

try:
    select_month = Select(driver.find_element_by_id('v-card-month'))
    select_month.select_by_visible_text(cart_month)
    select_year = Select(driver.find_element_by_id('v-card-year'))
    select_year.select_by_visible_text(cart_year)
except:
    pass

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("پرداخت") >=0 :
            button.click()
except:
    pass

time.sleep(3)
try:
    bodytxt = driver.find_element_by_tag_name('body').get_attribute('outerHTML')
    if bodytxt.find('در حال حاظر پاسخی از سمت بانک دریافت نشد') >=0 :
        buttons = driver.find_elements_by_tag_name("button")
        for button in buttons :
            btntxt = button.get_attribute('outerHTML')
            if btntxt.find("متوجه شدم") >=0 :
                button.click()
    if bodytxt.find('کافی نیست') >=0 :
        driver.quit()
except:
    pass

time.sleep(3)

try:
    # mli
    Input = driver.find_element_by_id("cf_melliCode")
    Input.clear()
    Input.send_keys('3510249917')
except:
    pass

try:
    buttons = driver.find_elements_by_tag_name("button")
    for button in buttons :
        btntxt = button.get_attribute('outerHTML')
        if btntxt.find("پرداخت") >=0 :
            button.click()
except:
    pass

time.sleep(3)
try:
    bodytxt = driver.find_element_by_tag_name('body').get_attribute('outerHTML')
    if bodytxt.find('کافی نیست') >=0 :
        driver.quit()
except:
    pass


درگاه پرداخت مسقیم – جیب گیت


C:\Users\farhad\AppData\Local\Temp\ipykernel_5524\214585697.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")
C:\Users\farhad\AppData\Local\Temp\ipykernel_5524\214585697.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  Inputs = driver.find_elements_by_tag_name("input")
C:\Users\farhad\AppData\Local\Temp\ipykernel_5524\214585697.py:36: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_tag_name("button")
C:\Users\farhad\AppData\Local\Temp\ipykernel_5524\214585697.py:46: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  Inputs = driver.find_elements_by_tag_name("input")
C:\Users\farhad\AppData\Local\Temp\ipykernel_5524\214585697.py:58: DeprecationWarning: find_element_by_* commands are dep

# Master Pay

In [82]:
Payment_gateway = driver.title

In [83]:
while True :
    active_tab = driver.find_element_by_class_name('pg-tab-active').get_attribute('outerHTML')

    if active_tab.find('ثبت نام') >= 0:
        try:
            button_label = driver.find_element_by_xpath("//*[text()='متوجه شدم']/../..")
            button = button_label.find_element_by_tag_name("button")
            button.click()
        except:
            pass

        try:
            # شماره کارت
            input_label = driver.find_element_by_xpath("//*[text()='شماره کارت:']/../..")
            Input = input_label.find_element_by_tag_name("input")
            Input.clear()
            Input.send_keys(cart_id)
        except:
            pass

        try:
            # تلفن همراه 
            input_label = driver.find_element_by_xpath("//*[text()='تلفن همراه:']/../..")
            Input = input_label.find_element_by_tag_name("input")
            Input.clear()
            Input.send_keys('0' + mobile_num)
        except:
            pass

        try:
            # مرحله بعد 
            buttons = driver.find_elements_by_tag_name("button")
            for button in buttons:
                if 'مرحله بعد' in button.get_attribute('outerHTML'):
                    button.click()
        except:
            pass

    if active_tab.find('تاییدیه') >= 0:
        smscod = input()
        # sms کد ارسال شد به: 
        Input = driver.find_element_by_tag_name("input")
        Input.clear()
        Input.send_keys(smscod)

        # مرحله بعد 
        buttons = driver.find_elements_by_tag_name("button")
        for button in buttons:
            if 'مرحله بعد' in button.get_attribute('outerHTML'):
                button.click()

    if active_tab.find('اطلاعات پرداخت') >= 0:

        try:
            inputs = driver.find_elements_by_tag_name("input")
            for inp in inputs:
                if 'cvv2' in inp.get_attribute('outerHTML'):
                    inp.clear()
                    inp.send_keys(cart_cvv2)
            for inp in inputs:
                if 'ماه' in inp.get_attribute('outerHTML'):
                    inp.clear()
                    inp.send_keys(cart_month)
            for inp in inputs:
                if 'سال' in inp.get_attribute('outerHTML'):
                    inp.clear()
                    inp.send_keys('05')

            input_label = driver.find_element_by_xpath("//*[text()='دریافت رمز']/..")
            Input = input_label.find_element_by_tag_name("input")
            Input.clear()
            Input.send_keys(cart_pass2)

            buttons = driver.find_elements_by_tag_name("button")
            for button in buttons:
                if 'پرداخت' in button.get_attribute('outerHTML'):
                    button.click()                    
        except:
            pass
        time.sleep(15)
        try:
            paystate = driver.find_element_by_tag_name('body').get_attribute('outerHTML').find('کافی نیست')
            if paystate >= 0:
                break
        except:
            pass

#         try:
#             buttons = driver.find_elements_by_tag_name("button")
#             for button in buttons:
#                 if 'بستن' in button.get_attribute('outerHTML'):
#                     button.click()
#                      break
#         except:
#             pass
    
    


C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3055547539.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  active_tab = driver.find_element_by_class_name('pg-tab-active').get_attribute('outerHTML')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3055547539.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button_label = driver.find_element_by_xpath("//*[text()='متوجه شدم']/../..")
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3055547539.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_label = driver.find_element_by_xpath("//*[text()='شماره کارت:']/../..")
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\3055547539.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_label = driver.find_element_by_xpath("//*[text()='تلفن همراه:']/../..")
C:\Users\f

# بانک

In [98]:
# s=Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(executable_path="chromedriver.exe")
# driver = webdriver.Chrome(service=s)
# driver.set_window_position(676,0)
# driver.set_window_size(697,735)
driver.get('https://ib.bpi.ir/')
c_url = driver.current_url
while c_url == driver.current_url :
    try :
        driver.find_element_by_xpath('//*[@id="AgreementSecureKeyboard"]/div/div[2]/label').click()

        Username = driver.find_element_by_xpath('//*[@id="username"]')
        Username.clear()
        Username.send_keys('15190717')

        Password = driver.find_element_by_xpath('//*[@id="password"]')
        Password.clear()
        Password.send_keys('farhadF72pass')
        driver.find_element_by_xpath('//*[@id="imageBtn"]').click()

        time.sleep(3)
        security_captcha_img = driver.find_element_by_xpath('//*[@id="captchaImg"]/img')

        with open('security_captcha.png', 'wb') as file:
            file.write(security_captcha_img.screenshot_as_png)
        ckeys = imagetostr('security_captcha.png')

        CaptchaTxt = driver.find_element_by_xpath('//*[@id="captchaTxt"]')
        CaptchaTxt.clear()
        CaptchaTxt.send_keys(ckeys)


        driver.find_element_by_xpath('//*[@id="btnLogin"]').click()
    except :
        pass

    
driver.get('https://ib.bpi.ir/CardForms/TransactionCardReport.aspx')


select = Select(driver.find_element_by_xpath('//*[@id="ctl00_activePage_ddlCard"]'))
select.select_by_visible_text('5022291310514847')

# set date for serch

jdate = gregorian_to_jalali(creat_at.year, creat_at.month, creat_at.day)

# from

fromday = driver.find_element_by_xpath('//*[@id="ctl00_activePage_startDate_day"]')
fromday.clear()
fromday.send_keys(jdate[2])

fromday = driver.find_element_by_xpath('//*[@id="ctl00_activePage_startDate_mounth"]')
fromday.clear()
fromday.send_keys(jdate[1])

fromday = driver.find_element_by_xpath('//*[@id="ctl00_activePage_startDate_year"]')
fromday.clear()
fromday.send_keys(jdate[0])
# to
day = driver.find_element_by_xpath('//*[@id="ctl00_activePage_endDate_day"]')
day.clear()
day.send_keys(jdate[2])

day = driver.find_element_by_xpath('//*[@id="ctl00_activePage_endDate_mounth"]')
day.clear()
day.send_keys(jdate[1])

day = driver.find_element_by_xpath('//*[@id="ctl00_activePage_endDate_year"]')
day.clear()
day.send_keys(jdate[0])


driver.find_element_by_xpath('//*[@id="ctl00_activePage_btnSearch"]').click()


# get table data 
tbody = driver.find_element_by_xpath('//*[@id="ctl00_activePage_grdCardTransactionsList"]').find_element_by_tag_name('tbody')

rows = tbody.find_elements_by_tag_name('tr') # get all of the rows in the table

Transaction = {}
for row in rows :
    cols = row.find_elements_by_tag_name('td')
    Transaction[cols[0].text] = {'time':cols[1].text,'value':cols[4].text,'Destination_card':cols[5].text,'Issue_Tracking':cols[6].text}

# ?
lastitem = [i for i in Transaction.keys()][-1]
Destination_card = Transaction[lastitem]['Destination_card']
transaction_value = Transaction[lastitem]['value']
Issue_Tracking = Transaction[lastitem]['Issue_Tracking']
date_time = Transaction[lastitem]['time'].split('\n')
jdate = [int(i) for i in date_time[0].split('/')]
jdate = jalali_to_gregorian(jdate[0], jdate[1], jdate[2])
miladi_datetime = str(jdate[0])+'-'+str(jdate[1])+'-'+str(jdate[2])+' '+date_time[1]

#

driver.get('https://ib.bpi.ir/Transfers/CardToCard.aspx')
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_sourceInputPH_CardControl_txtCVV2_txtContent"]').send_keys('185')
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_sourceInputPH_CardControl_txtMonthExpireDate_txtContent"]').send_keys('12')
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_sourceInputPH_CardControl_txtYearExpireDate_txtContent"]').send_keys('05')
# 
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_simpleDestinationPH_DestCardControl_CardNumberControl_txtCardPart_1"]').send_keys(Destination_card)
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_simpleDestinationPH_amount_txtContent"]').send_keys('10000')
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_SendOtpPlaceHolder_sendOtpControl_txtcardPass_txtContent"]').send_keys('4241930')
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_btnSeeDatail"]').click()

# get subject name
subject_name = driver.find_element_by_xpath('/html/body/form/div[4]/div/div[2]/div[2]/div[3]/div[4]/div/div/div[7]/div[2]/span').text
# 
BankName = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_activePage_TransferDetailFormPH_lblBankName"]').text

driver.quit()



con = sqlite3.connect('db.sqlite3')
cur = con.cursor()

res = cur.execute('select id from betscan_cart where cartid = '+ Destination_card)
if len(res.fetchall()) <= 0:
    sql1 = "insert into betscan_cart (cartid, customer_name, creat_at, bank_name) values ('{}','{}','{}','{}')".format(Destination_card,subject_name,miladi_datetime,BankName)
    cur.execute(sql1)

sql = "select id from betscan_cart where cartid = '{}' ".format(Destination_card)
res = cur.execute(sql)
Cart_id = res.fetchone()[0]

sql = "select id from betscan_website where address = '{}' ".format(website_url)
res = cur.execute(sql)
Website_id = res.fetchone()[0]

res = cur.execute("select id from betscan_records where Cart_id = '{}' and Website_id = '{}' and creat_at = '{}' ".format(Cart_id,Website_id,miladi_datetime))
if len(res.fetchall()) <= 0:
    sql2 = "insert into betscan_records(Cart_id, Website_id, creat_at, transaction_value, Issue_Tracking, Payment_gateway) values ('{}','{}','{}','{}','{}','{}')".format(Cart_id, Website_id, miladi_datetime, transaction_value, Issue_Tracking, Payment_gateway)
    cur.execute(sql2)
    


con.commit()
con.close()

C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\2208721480.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="AgreementSecureKeyboard"]/div/div[2]/label').click()
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\2208721480.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Username = driver.find_element_by_xpath('//*[@id="username"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\2208721480.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Password = driver.find_element_by_xpath('//*[@id="password"]')
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\2208721480.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="imageBtn"]').click()
C:\Users\farhad\AppData\Local\Temp\ipykernel_4784\2208